# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [4]:
import riak

In [5]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

¿Está Riak disponible? True


In [6]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

"2.1.7-0-gbd8e312"


## Insertar valores

In [7]:
bucket = cliente.bucket("bucket_name")

In [8]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [9]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [10]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [11]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [12]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [13]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

key_one: 1
key_two: Dos
key_three: 
{'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}


In [14]:
!curl http://localhost:8098/riak/bucket_name/key_one

1

In [15]:
!curl http://localhost:8098/riak/bucket_name/key_two

"Dos"

In [16]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

{
  "val1": 1,
  "val2": "Dos",
  "val3": [
    1,
    2,
    3,
    4,
    5
  ]
}


http://localhost:8098/riak/images/image_key

## Índices

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [17]:
import pandas as pd
import json
import uuid
import random

bucket = cliente.bucket("ejemplo_indices")

col_Ciudad = ["Madrid", "Madrid", "Barcelona", "Sevilla", "Madrid"]

for i in range(0, 5):
    key = str(i+1)
    
    valor = {"value" : random.randint(0,255),
             "key_aux" :  uuid.uuid4().hex,
             "ciudad" : col_Ciudad[i]}

    riak_key = bucket.new(key, data = valor)
    riak_key.add_index('idx_ciudad_bin', valor["ciudad"])
    riak_key.add_index('idx_key_aux_bin', valor["key_aux"])
    riak_key.store()
    
    print("key", key, ":", valor)


key 1 : {'value': 246, 'key_aux': '1e5522d7a6984bde9527c77d8917ee5e', 'ciudad': 'Madrid'}
key 2 : {'value': 139, 'key_aux': '932b5d683c1f4fe58529fdfac28fa6d9', 'ciudad': 'Madrid'}
key 3 : {'value': 199, 'key_aux': '0deaf114889c42acb9b8e31090b968c6', 'ciudad': 'Barcelona'}
key 4 : {'value': 211, 'key_aux': '71884ce8a97841d98edd95bf41f9a114', 'ciudad': 'Sevilla'}
key 5 : {'value': 65, 'key_aux': '31aab13d09a547258e2046a6bb22720c', 'ciudad': 'Madrid'}


Cargamos la información en Pandas

In [18]:
rows = []
for keys in bucket.stream_keys():
    for key in keys:
        rows.append(bucket.get(key).data)

df = pd.read_json(json.dumps(rows))
df

,value,key_aux,ciudad
0,139,932b5d683c1f4fe58529fdfac28fa6d9,Madrid
1,246,1e5522d7a6984bde9527c77d8917ee5e,Madrid
2,211,71884ce8a97841d98edd95bf41f9a114,Sevilla
3,199,0deaf114889c42acb9b8e31090b968c6,Barcelona
4,65,31aab13d09a547258e2046a6bb22720c,Madrid


Ahora vamos a localizar las claves de "Madrid" a través del ínidice

In [19]:
keys = bucket.get_index("idx_ciudad_bin", 'Madrid')
for key in keys.results:
    print(bucket.get(key).data)

{'value': 65, 'key_aux': '31aab13d09a547258e2046a6bb22720c', 'ciudad': 'Madrid'}
{'value': 139, 'key_aux': '932b5d683c1f4fe58529fdfac28fa6d9', 'ciudad': 'Madrid'}
{'value': 246, 'key_aux': '1e5522d7a6984bde9527c77d8917ee5e', 'ciudad': 'Madrid'}


Ahora vamos a localizar una clave auxiliar a través de su ínidice

In [25]:
key_aux = df.at[0, 'key_aux']
print(key_aux)

932b5d683c1f4fe58529fdfac28fa6d9


In [26]:
keys = bucket.get_index("idx_key_aux_bin", key_aux)
for key in keys.results:
    print(bucket.get(key).data)

## Listado de Buckets

In [27]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

Bucket <RiakBucket 'movimientos'>
Bucket <RiakBucket 'users'>
Bucket <RiakBucket 'tweets'>
Bucket <RiakBucket 'images'>
Bucket <RiakBucket 'hashtags'>
Bucket <RiakBucket 'ejemplo_indices'>
Bucket <RiakBucket 'people'>
Bucket <RiakBucket 'bucket_name'>


## Listado de valores en un Bucket

In [28]:
for keys in bucket.stream_keys():
    print("Keys: ", keys)
    for key in keys:
        value = bucket.get(key).data
        print('Key %s: %s' % (key, value))

Keys:  ['key_two']
Key key_two: Dos
Keys:  ['key_three']
Key key_three: {'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}


In [29]:
for key in bucket.get_keys():
    value = bucket.get(key).data
    print('Key %s: %s' % (key, value))

Key key_three: {'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}
Key key_two: Dos
